# EDA La Liga Dataset

Este notebook realiza un análisis exploratorio del dataset longitudinal generado a partir de FBref y Transfermarkt.\n
Se presentan al menos diez insights relevantes, apoyados con tablas y gráficos generados con **matplotlib** (no se utiliza seaborn).

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

# Cargar el dataset longitudinal generado por el pipeline
data_path = Path('data/final/fbref_tm_laliga_longitudinal.csv')
if not data_path.exists():
    raise FileNotFoundError('No se encuentra el dataset final. Ejecuta el pipeline previamente.')
df = pd.read_csv(data_path)
df.head()

## 1. Distribución del valor de mercado por posición

Boxplot de `mv_millions` para cada posición (filtrando valores extremos si fuera necesario).

In [ ]:
positions = df['position'].dropna().unique()
fig, ax = plt.subplots(figsize=(10, 6))
df.boxplot(column='mv_millions', by='position', ax=ax, vert=False)
ax.set_xlabel('Valor de mercado (millones €)')
ax.set_ylabel('Posición')
ax.set_title('Distribución de mv_millions por posición')
plt.suptitle('')
plt.show()

## 2. Evolución temporal del valor medio por club

Gráfico de líneas que muestra la evolución del valor medio de mercado por club a lo largo de las temporadas.

In [ ]:
# Calcular el valor medio por club y temporada
club_mean = df.groupby(['Season', 'club'])['mv_millions'].mean().reset_index()
# Seleccionar algunos clubes de ejemplo (los 5 con mayor valor medio global)
top_clubs = club_mean.groupby('club')['mv_millions'].mean().nlargest(5).index
fig, ax = plt.subplots(figsize=(10, 6))
for club in top_clubs:
    sub = club_mean[club_mean['club'] == club]
    ax.plot(sub['Season'], sub['mv_millions'], label=club)
ax.set_xlabel('Temporada')
ax.set_ylabel('Valor medio (millones €)')
ax.set_title('Evolución del valor medio por club (top 5)')
ax.legend()
plt.xticks(rotation=45)
plt.show()

## 3. Top 10 jugadores por crecimiento de valor inter-temporadas

Se calcula el cambio de valor de mercado de cada jugador entre temporadas consecutivas y se listan los 10 con mayor incremento.

In [ ]:
# Ordenar por jugador y temporada para calcular diferencias
df_sorted = df.sort_values(['Player', 'Season'])
df_sorted['mv_shift'] = df_sorted.groupby('Player')['mv_millions'].shift(1)
df_sorted['mv_change'] = df_sorted['mv_millions'] - df_sorted['mv_shift']
top_growth = df_sorted.nlargest(10, 'mv_change')[['Player', 'Season', 'mv_change']]
top_growth

## 4. Relación edad vs. valor de mercado

Se agrupan los jugadores por rangos de edad y se muestra el valor medio en cada rango.

In [ ]:
# Crear tramos de edad
bins = [15, 20, 25, 30, 35, 40, 45]
labels = ['15-20', '20-25', '25-30', '30-35', '35-40', '40-45']
df['age_group'] = pd.cut(df['Age'], bins=bins, labels=labels, right=False)
age_mean = df.groupby('age_group')['mv_millions'].mean()
age_mean.plot(kind='bar', figsize=(8, 5))
plt.xlabel('Rango de edad')
plt.ylabel('Valor medio (millones €)')
plt.title('Edad vs. valor de mercado')
plt.show()

## 5. Top nacionalidades por valor medio

Se agrupan los jugadores por su principal nacionalidad y se muestran las nacionalidades con mayor valor medio.

In [ ]:
# Extraer la primera nacionalidad si hay varias
df['main_nat'] = df['nationality'].apply(lambda x: x[0] if isinstance(x, list) and x else None)
nat_mean = df.groupby('main_nat')['mv_millions'].mean().dropna().nlargest(10)
nat_mean.plot(kind='bar', figsize=(8, 5))
plt.xlabel('Nacionalidad')
plt.ylabel('Valor medio (millones €)')
plt.title('Nacionalidades con mayor valor medio (Top 10)')
plt.show()

## 6. Outliers de valor vs. minutos jugados

Si el dataset incluye minutos o partidos jugados, se pueden detectar jugadores cuyo valor sea demasiado alto respecto a su tiempo en el campo.

In [ ]:
# Suponiendo que existe una columna 'Minutes' en df
if 'Minutes' in df.columns:
    df['value_per_min'] = df['mv_millions'] / df['Minutes']
    outliers = df.nlargest(10, 'value_per_min')[['Player', 'Season', 'mv_millions', 'Minutes', 'value_per_min']]
    outliers

## 7. Mapa de correlaciones de variables numéricas

Se calcula la matriz de correlación de variables numéricas y se muestra como un mapa de calor.

In [ ]:
num_cols = df.select_dtypes(include='number').columns
corr = df[num_cols].corr()
fig, ax = plt.subplots(figsize=(8, 6))
cax = ax.matshow(corr, cmap='viridis')
plt.xticks(range(len(num_cols)), num_cols, rotation=90)
plt.yticks(range(len(num_cols)), num_cols)
plt.colorbar(cax)
plt.title('Matriz de correlación')
plt.show()

## 8. Clubs con mayor revalorización neta por temporada

Se calcula la diferencia de valor total del club entre temporadas consecutivas.

In [ ]:
club_sum = df.groupby(['Season', 'club'])['mv_millions'].sum().reset_index()
club_sum_sorted = club_sum.sort_values(['club', 'Season'])
club_sum_sorted['sum_prev'] = club_sum_sorted.groupby('club')['mv_millions'].shift(1)
club_sum_sorted['delta'] = club_sum_sorted['mv_millions'] - club_sum_sorted['sum_prev']
top_delta = club_sum_sorted.nlargest(10, 'delta')[['Season', 'club', 'delta']]
top_delta

## 9. Posiciones con mayor volatilidad de valor

Se calcula la desviación estándar del valor de mercado por posición.

In [ ]:
volatility = df.groupby('position')['mv_millions'].std().dropna().nlargest(10)
volatility.plot(kind='barh', figsize=(8, 5))
plt.xlabel('Desviación estándar (millones €)')
plt.ylabel('Posición')
plt.title('Volatilidad del valor por posición (Top 10)')
plt.show()

## 10. Calidad de datos: % de nulos por columna clave

Se calcula el porcentaje de valores nulos en columnas clave y se evalúa su impacto potencial.

In [ ]:
cols_to_check = ['mv_millions', 'position', 'Age', 'Nation', 'club', 'Season']
null_pct = df[cols_to_check].isna().mean() * 100
+null_pct.plot(kind='bar', figsize=(8,5))
+plt.ylabel('% de valores nulos')
+plt.title('Porcentaje de nulos por columna clave')
+plt.show()

## Conclusiones y acciones

Aquí se resumen los principales hallazgos del análisis y se proponen acciones futuras, incluyendo qué variables podrían ser más relevantes en un modelo de predicción de revalorización.